# Connection status

`use_connection_status()` lets your components react to connectivity changes.
You can use it to display the connection status and inform users about the app state.

## Connection info

The hook returns a reactive status object with fields such as:

- `connected`: `True`, `False`, or `None` (unknown)
- `reason`: a short machine-readable reason
- `transport`: e.g. `jupyter-comm`, `standalone-http`, `websocket`
- `kernel_status`: current Jupyter kernel status when relevant
- `kernel_connection_status`: Jupyter kernel connection status when relevant
- `last_error`: last transport/send error if any

Whenever these values change, components using the hook re-render automatically.

## Examples

In [1]:
from pret_joy import Box, Button, Stack, Typography

from pret import component, create_store, use_connection_status, use_store_snapshot
from pret.react import h4, p, span

# Local-only store: never needs a backend connection.
local_state = create_store({"count": 0})

# Synced store: requires a live server/kernel counterpart to persist/synchronize updates.
synced_state = create_store({"count": 0}, sync=True)


def increment_local(event):
    local_state["count"] = local_state["count"] + 1


def increment_synced(event):
    synced_state["count"] = synced_state["count"] + 1


@component
def ConnectionStatusDemo():
    status = use_connection_status()
    local = use_store_snapshot(local_state)
    synced = use_store_snapshot(synced_state)

    # fmt: off
    return Stack(
        [
            h4("Connection status"),
            Box(
                span("connected:"), span(str(status["connected"])),
                span("reason:"), span(status["reason"]),
                span("transport:"), span(status["transport"]),
                span("kernel_status:"), span(status["kernel_status"]),
                span("kernel_connection_status:"), span(status["kernel_connection_status"]),
                span("last_error:"), span(status["last_error"]),
                style={
                    "display": "grid",
                    "gridTemplateColumns": "1fr 1fr",
                }
            ),
            h4("Update behavior"),
            p(f"Local store count (always local): {local['count']}"),
            Button("Increment local store", on_click=increment_local),
            p(f"Synced store count (requires backend): {synced['count']}"),
            Button("Increment synced store", on_click=increment_synced),
            p("When disconnected, synced updates are rolled back. Local updates still apply."),
        ],
        spacing=1.25,
        sx={"m": 1, "width": "100%"},
    )
    # fmt: off


ConnectionStatusDemo()

Here is how a simple status message can be customized depending on the connection status.

In [2]:
from pret_joy import Stack, Typography

from pret import component, use_connection_status


@component
def ConnectionAwareMessage():
    status = use_connection_status()

    connected = status["connected"] is True
    text = (
        "Remote sync is available."
        if connected
        else "Running without remote sync (offline/local-only mode)."
    )

    return Stack(
        [
            Typography("Connection-aware message", level="title-lg"),
            Typography(
                text,
                level="body-md",
                color=("success" if connected else "warning"),
            ),
        ],
        spacing=1,
        sx={"m": 1, "width": "100%"},
    )


ConnectionAwareMessage()